# Rijnland

This script adds a new column "peilgebied_cat" and makes sure the peilgebieden allign with the HWS layer (Daniel):
- peilgebied_cat = 0 -> peilgebied
- peigelbied_cat = 1 -> RHWS (boezem)
- peilgebied_cat = 2 -> NHWS 

In [ ]:
import geopandas as gpd
import numpy as np

%load_ext autoreload
%autoreload 2

from general_functions import *

## Rijnland

In [ ]:
# define relative paths
waterschap = "Rijnland"

data_path = f"../projects/4750_30/Data_postprocessed/Waterschappen/{waterschap}/{waterschap}.gpkg"

# Waterschaps boundaries
grens_path = "../projects/4750_30/Data_overig/Waterschapsgrenzen/Waterschapsgrenzen.geojson"
# Hoofdwatersysteem boundaries
hws_path = "../projects/4750_30/Data_overig/HWS/krw_basins_vlakken.gpkg"
# Buffer boundaries
buffer_path = "../projects/4750_30/Data_overig/HWS/hws_buffer_rijnland.gpkg"
# Output folder
output_folder = f"./Waterschappen/{waterschap}"

### Load Files

In [ ]:
# Load HHNK files
Rijnland = read_gpkg_layers(
    gpkg_path=data_path,
    variables=[
        "stuw",
        "gemaal",
        "hydroobject",
        "duikersifonhevel",
        "peilgebied",
        "streefpeil",
    ],
)
Rijnland["peilgebied"] = Rijnland["peilgebied"].to_crs("EPSG:28992")

# Load waterschap boundaries
gdf_grens = gpd.read_file(grens_path)
gdf_grens = gdf_grens.to_crs("EPSG:28992")
gdf_grens = gdf_grens.set_index("waterschap")

# Load hws
gdf_hws = gpd.read_file(hws_path)

# Load buffer
gdf_buffer = gpd.read_file(buffer_path)

## Select waterschap boundaries and clip hws layer

In [ ]:
# Select boundaries HH Amstel, Gooi en Vecht
gdf_grens = gdf_grens.loc[["HH van Rijnland"]]

# Use waterschap boudnaries to clip HWS layer
gdf_hws = gpd.overlay(gdf_grens, gdf_hws, how="intersection")

## Peilgebied and HWS layer overlap:
1. Identify the overlapping areas
2. Clip
3. Calculate overlapping area percentage
4. Filter

In [ ]:
# Step 1: Identify the Overlapping Areas and clip
overlaps = gpd.overlay(Rijnland["peilgebied"], gdf_hws, how="intersection", keep_geom_type=True)

# # Step 2: Subtract Overlapping Areas from the original polygons in each DataFrame
non_overlapping_peilgebied = gpd.overlay(Rijnland["peilgebied"], overlaps, how="difference", keep_geom_type=True)
overlaps = gpd.overlay(non_overlapping_peilgebied, gdf_hws, how="intersection", keep_geom_type=False)

# Step 3: Calculate Area Percentages
# Calculate the area of overlaps
overlaps["overlap_area"] = overlaps.area

# Step 4: Filter based on area Area Percentages
minimum_area = 20000
print(f"Number of overlapping shapes without filter: {len(overlaps)}")
overlap_ids = overlaps.loc[overlaps["overlap_area"] > minimum_area]
overlap_ids = overlap_ids.globalid.to_list()
print(f"Number of overlapping shapes with filter: {len(overlap_ids)}")

## Create peilgebied_cat column

In [ ]:
# Add occurence to geodataframe
peilgebieden_cat = []

for index, row in Rijnland["peilgebied"].iterrows():
    if row.code == "dummy_code_peilgebied_18207":
        peilgebieden_cat.append(1)
        print("yes")
    elif row.code == "dummy_code_peilgebied_18322":
        peilgebieden_cat.append(1)
    elif row.code == "dummy_code_peilgebied_18155":
        peilgebieden_cat.append(1)
    elif row.code == "dummy_code_peilgebied_18161":
        peilgebieden_cat.append(1)
    elif row.code == "dummy_code_peilgebied_19451":
        peilgebieden_cat.append(2)
    else:
        peilgebieden_cat.append(0)

# Add new column and drop old HWS_BZM column
Rijnland["peilgebied"]["peilgebied_cat"] = peilgebieden_cat

## Add rhws to ['peilgebied','streefpeil']

In [ ]:
# update peilgebied dict key
gdf_rhws["globalid"] = "dummy_globalid_rhws_" + gdf_rhws.index.astype(str)
gdf_rhws["code"] = "dummy_code_nhws_" + gdf_rhws.index.astype(str)
gdf_rhws["nen3610id"] = "dummy_nen3610id_rhws_" + gdf_rhws.index.astype(str)
gdf_rhws["peilgebied_cat"] = 1

gdf_rhws = gdf_rhws[["globalid", "code", "nen3610id", "peilgebied_cat", "geometry"]]

Rijnland["peilgebied"] = pd.concat([gdf_rhws, AVG["peilgebied"]])

In [ ]:
# Create boezem streefpeil layer
streefpeil_hws = pd.DataFrame()
streefpeil_hws["waterhoogte"] = [np.nan] * len(gdf_rhws)
streefpeil_hws["globalid"] = "dummy_globalid_rhws_" + gdf_rhws.index.astype(str)
streefpeil_hws["geometry"] = [None] * len(gdf_rhws)

Rijnland["streefpeil"] = pd.concat([streefpeil_hws, Rijnland["streefpeil"]])
Rijnland["streefpeil"] = gpd.GeoDataFrame(Rijnland["streefpeil"])

## Add nhws to ['peilgebied','streefpeil']

In [ ]:
# update peilgebied dict key
gdf_hws["globalid"] = "dummy_globalid_nhws_" + gdf_hws.index.astype(str)
gdf_hws["code"] = "dummy_code_nhws_" + gdf_hws.index.astype(str)
gdf_hws["nen3610id"] = "dummy_nen3610id_nhws_" + gdf_hws.index.astype(str)
gdf_hws["peilgebied_cat"] = 2

gdf_hws = gdf_hws[["globalid", "code", "nen3610id", "peilgebied_cat", "geometry"]]

Rijnland["peilgebied"] = pd.concat([gdf_hws, Rijnland["peilgebied"]])

In [ ]:
# Create boezem streefpeil layer
streefpeil_hws = pd.DataFrame()
streefpeil_hws["waterhoogte"] = [np.nan] * len(gdf_hws)
streefpeil_hws["globalid"] = "dummy_globalid_nhws_" + gdf_hws.index.astype(str)
streefpeil_hws["geometry"] = [None] * len(gdf_hws)

Rijnland["streefpeil"] = pd.concat([streefpeil_hws, Rijnland["streefpeil"]])
Rijnland["streefpeil"] = gpd.GeoDataFrame(Rijnland["streefpeil"])

### Select waterschap boundaries

In [ ]:
# Select boundaries HH Amstel, Gooi en Vecht
gdf_grens = gdf_grens.loc[["HH van Rijnland"]]

### Create inverse layer

In [ ]:
# Remove mixed geomtypes (lines)
data = []

for index, row in Rijnland["peilgebied"].iterrows():
    #     print(row.geometry.geom_type)
    if row.geometry.geom_type != "LineString":
        data.append(row)

Rijnland["peilgebied"] = gpd.GeoDataFrame(pd.concat(data, axis=1, ignore_index=True)).transpose()
Rijnland["peilgebied"] = Rijnland["peilgebied"].set_geometry("geometry")
Rijnland["peilgebied"] = Rijnland["peilgebied"].set_crs("EPSG:28992")

In [ ]:
# Select inverse of peilgebied
gdf_boezem_out = gpd.overlay(gdf_grens, Rijnland["peilgebied"].dissolve(), how="symmetric_difference")

In [ ]:
# Store unfiltered layer
gdf_boezem_out.to_file(f"{output_folder}/boezem_unfiltered_{waterschap}.gpkg")

In [ ]:
# Create separate polygons
gdf_boezem_out = gdf_boezem_out.explode()

### Calculate area of polygons and filter

In [ ]:
# Calculate area of polygons
areas = []

for index, row in gdf_boezem_out.iterrows():
    areas.append(row.geometry.area)

gdf_boezem_out["area"] = areas

In [ ]:
# filter based on area of polygons
gdf_boezem_out.sort_values(by="area").iloc[[-1]].to_file(f"{output_folder}/boezem_filter_lvl_1_{waterschap}.gpkg")
gdf_boezem_out.sort_values(by="area").iloc[[-2]].to_file(f"{output_folder}/boezem_filter_lvl_2_{waterschap}.gpkg")

In [ ]:
# Store peilgebieden that do not connect properly
gdf_boezem_out.sort_values(by="area").iloc[:-1].to_file(f"{output_folder}/niet_goed_aansluitend_{waterschap}.gpkg")

### Add boezem when peilgebied is part of dm_netwerk

In [ ]:
# Load Boezem network file (DM_netwerk)
gdf_dm_netwerk = gpd.read_file(dm_netwerk_path)

In [ ]:
# Select the peilgebieden that intersect with DM-netwerk
gdf = gpd.overlay(Rijnland["peilgebied"], gdf_dm_netwerk, how="intersection")

### Add HWS_BZM flag to boezem polygons

In [ ]:
# Add occurence to geodataframe
boezems = []

for index, row in Rijnland["peilgebied"].iterrows():
    if row.nen3610id in gdf.nen3610id.values:
        boezems.append(True)
    else:
        boezems.append(False)

Rijnland["peilgebied"]["HWS_BZM"] = boezems

In [ ]:
for key in Rijnland.keys():
    print(key)
    Rijnland[str(key)].to_file(f"{output_folder}/{waterschap}_bzm.gpkg", layer=str(key), driver="GPKG")

### Merge boezem and peilgebied layers

In [ ]:
# Select globalids of boezem polygons
bzm_id = Rijnland["peilgebied"].loc[Rijnland["peilgebied"]["HWS_BZM"]].globalid

# Match globalids with streefpeil layer globalids
bzm_waterhoogte = Rijnland["streefpeil"].loc[Rijnland["streefpeil"]["globalid"].isin(bzm_id)]

print(len(bzm_id))
print(len(bzm_waterhoogte))

In [ ]:
# Create boezem layer
boezem = gdf_boezem_out.sort_values(by="area").iloc[[-1]]

boezem["code"] = "dummy_code_999999"
boezem["globalid"] = "dummy_globalid_999999"
boezem["nen3610id"] = "dummy_nen3610id_peilgebied_999999"
boezem["HWS_BZM"] = True
boezem = boezem[["code", "globalid", "nen3610id", "HWS_BZM", "geometry"]]

# Create boezem streefpeil layer
streefpeil_bzm = pd.DataFrame()
streefpeil_bzm["waterhoogte"] = [None]
streefpeil_bzm["globalid"] = ["dummy_globalid_999999"]
streefpeil_bzm["geometry"] = [None]

In [ ]:
# Merge boezem layer with peilgebieden
Rijnland["peilgebied"] = gpd.GeoDataFrame(pd.concat([boezem, Rijnland["peilgebied"]], ignore_index=True))
Rijnland["streefpeil"] = gpd.GeoDataFrame(pd.concat([streefpeil_bzm, Rijnland["streefpeil"]], ignore_index=True))

In [ ]:
for key in Rijnland.keys():
    print(key)
    Rijnland[str(key)].to_file(f"{output_folder}/{waterschap}.gpkg", layer=str(key), driver="GPKG")